In [6]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project='your-project-id')
import pprint

In [2]:

# Load urban boundaries (GAUL dataset)
urban = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# Filter Tehran boundary
Tehran = urban.filter(ee.Filter.eq('ADM1_NAME', 'Tehran'))  # Note: 'Tehran' is the correct spelling in GAUL
geometry = Tehran.geometry()

In [7]:
# Load Sentinel-2 imagery
s2 = ee.ImageCollection("COPERNICUS/S2")

# Apply filters (cloud cover, date, and location)
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
             .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
             .filter(ee.Filter.bounds(geometry))

# Compute median composite
image = filtered.median()
# Inspect the image object in the Console (printed as JSON-like structure)
pprint.pprint(image.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'double',
                          'type': 'PixelType'},
            'id': 'B1'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'double',
                          'type': 'PixelType'},
            'id': 'B2'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'double',
                          'type': 'PixelType'},
            'id': 'B3'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'max': 65535,
                          'min': 0,
      

In [8]:
# Calculate NDVI (B8 = NIR, B4 = Red)
ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')

# Calculate MNDWI (B3 = Green, B11 = SWIR)
mndwi = image.normalizedDifference(['B3', 'B11']).rename('mndwi')

# Calculate SAVI (with reflectance scaling)
savi = image.expression(
    '1.5 * ((NIR - RED) / (NIR + RED + 0.5))', {
      'NIR': image.select('B8').multiply(0.0001),
      'RED': image.select('B4').multiply(0.0001)
}).rename('savi')

In [11]:
# Create a map using geemap
Map = geemap.Map()
# Center the map on Tehran
Map.centerObject(geometry, 8)

# Visualization parameters
rgbVis = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
ndviVis = {'min': 0, 'max': 1, 'palette': ['white', 'green']}
ndwiVis = {'min': 0, 'max': 0.5, 'palette': ['white', 'blue']}

# Display layers
Map.addLayer(image.clip(geometry), rgbVis, 'Image')
Map.addLayer(mndwi.clip(geometry), ndwiVis, 'mndwi')
Map.addLayer(savi.clip(geometry), ndviVis, 'savi')
Map.addLayer(ndvi.clip(geometry), ndviVis, 'ndvi')
Map

Map(center=[35.66879776649009, 51.47536620026898], controls=(WidgetControl(options=['position', 'transparent_b…